# Ejercicio Block Cipher
Diego Andrés Morales Aquino
21762


## 1. Generación una función cifrado y descifrado DES 

Código realizado con el apoyo del modelo GPT 3.5 <br>
Prompt disponible en: https://chatgpt.com/share/67ca43b1-7c64-8009-b4f7-8467992c94ff

In [21]:
from Crypto.Cipher import DES
from Crypto.Random import get_random_bytes
import binascii

In [ ]:

def pad(text):
    """
    Agregar padding. El padding está compuesto por el char que corresponde a la longitud del padding.
    """
    pad_length = 8 - len(text) % 8
    padding = chr(pad_length) * pad_length
    return text + padding

def des_encrypt(plain_text):

    key = get_random_bytes(8)
    padded_text = pad(plain_text)
    
    # Cifrado DES en modo ECB
    cipher = DES.new(key, DES.MODE_ECB)
    encrypted_text = cipher.encrypt(padded_text.encode())
    
    # Retornar llave y cifrado en hexa
    return binascii.hexlify(key).decode(), binascii.hexlify(encrypted_text).decode()

In [ ]:
def unpad(padded_text):
    """
    Obtener la longitud del padding a partir del ascii del último char y eliminarlo.
    """
    pad_length = ord(padded_text[-1])
    return padded_text[:-pad_length]

def des_decrypt(key_hex, encrypted_text_hex):

    # hex a bytes
    key = binascii.unhexlify(key_hex)
    encrypted_text = binascii.unhexlify(encrypted_text_hex)
    
    # Desencriptar con DES
    cipher = DES.new(key, DES.MODE_ECB)
    decrypted_text = cipher.decrypt(encrypted_text)
    
    # Eliminar el padding
    original_text = unpad(decrypted_text.decode())
    
    return original_text

In [24]:
# Encriptar texto
plain_text = "Mensaje de Diego Morales."
key, encrypted_text = des_encrypt(plain_text)

print(f"Llave generada: {key}")
print(f"Texto cifrado: {encrypted_text}")

Llave generada: 9991d3e6a2eb2774
Texto cifrado: b5645534d928298ca52ef2f8fb962b9fb6ff391212401389382b1a7eeaa31a09


In [25]:
# Desencriptar el texto
original_text = des_decrypt(key, encrypted_text)

print(f"Texto original: {original_text}")

Texto original: Mensaje de Diego Morales.


## 2. Generación una función cifrado y descifrado 3DES

In [3]:
from Crypto.Cipher import DES3
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
import base64

In [8]:
def encrypt_3des_cbc(message: str) -> str:
    # Clave de 24 bytes
    key = get_random_bytes(24)
    
    # Generar un vector de inicialización
    iv = get_random_bytes(8)
    
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    
    # Convertir a bytes y hacer padding hasta que sea múltiplo del tamaño de bloque
    message_bytes = message.encode('utf-8')
    padded_message = pad(message_bytes, DES3.block_size)
    
    # Cifrar
    encrypted_bytes = cipher.encrypt(padded_message)
    
    # Retornar la clave, IV y el mensaje cifrado en base64
    return base64.b64encode(key + iv + encrypted_bytes).decode('utf-8')

In [ ]:
def decrypt_3des_cbc(encrypted_message: str) -> str:
    # Decodificar desde base64
    encrypted_data = base64.b64decode(encrypted_message)
    
    # Viene en formato key (24b)+iv(8b)+encrypted_message
    key = encrypted_data[:24]
    iv = encrypted_data[24:32]
    encrypted_bytes = encrypted_data[32:]
    
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    
    # Desencriptar y quitar el padding
    decrypted_padded = cipher.decrypt(encrypted_bytes)
    decrypted_message = unpad(decrypted_padded, DES3.block_size)
    
    return decrypted_message.decode('utf-8')

In [10]:
print("Mensaje crifrado con 3DES:")
mensaje_cifrado = encrypt_3des_cbc("contraseña: 123456")
print(mensaje_cifrado)

Mensaje crifrado con 3DES:
FMW/3PONCafu1FHia+Ob3VCGyG2TTbijVI5mLEZLGSsUdRtws4Ioz6+SEBuLm7mVpSqSlHe6dLA=


In [11]:
print("Mensaje descifrado con 3DES:")
mensaje_descifrado = decrypt_3des_cbc(mensaje_cifrado)
print(mensaje_descifrado)


Mensaje descifrado con 3DES:
contraseña: 123456


## 3. Generación una función cifrado y descifrado AES con CBC Y ECB

In [ ]:
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import numpy as np
import os

In [ ]:
def encrypt_image(img_path, mode="ECB"):
    # Cargar imagen y convertir a RGB
    img = Image.open(img_path).convert("RGB")
    img_array = np.array(img)
    img_bytes = img_array.tobytes()
    
    # Generar key
    key = get_random_bytes(16)
    
    if mode == "CBC":
        iv = get_random_bytes(16)  # Generar vecotr de inicialización
        cipher = AES.new(key, AES.MODE_CBC, iv)
    else:
        cipher = AES.new(key, AES.MODE_ECB)
        iv = None  # No se usa vector de inicialización en ECB
    
    # Añadir padding y cifrar
    padded_bytes = pad(img_bytes, AES.block_size)
    encrypted_bytes = cipher.encrypt(padded_bytes)
    
    # Guardar como imagen
    encrypted_array = np.frombuffer(encrypted_bytes, dtype=np.uint8)
    encrypted_array = encrypted_array[:img_array.size].reshape(img_array.shape)  
    encrypted_img = Image.fromarray(encrypted_array)
    
    output_file = f"encrypted_{mode}_{os.path.basename(img_path)}"
    encrypted_img.save(output_file)
    
    return key, iv, img.size, output_file

In [71]:
def decrypt_image_from_png(encrypted_png_path, key, iv=None, mode="ECB", img_size=(0, 0)):
    # Cargar imagen cifrada
    encrypted_img = Image.open(encrypted_png_path).convert("RGB")
    encrypted_array = np.array(encrypted_img)
    encrypted_bytes = encrypted_array.tobytes()
    
    # Añadir padding si no es múltiplo del tamaño de bloque
    block_size = AES.block_size
    if len(encrypted_bytes) % block_size != 0:
        padding_length = block_size - (len(encrypted_bytes) % block_size)
        encrypted_bytes += b'\0' * padding_length
    
    # crear objeto para cifrar según el modo
    if mode == "CBC":
        cipher = AES.new(key, AES.MODE_CBC, iv)
    else:
        cipher = AES.new(key, AES.MODE_ECB)
    
    try:
        # Descifrar y eliminar padding
        decrypted_bytes = unpad(cipher.decrypt(encrypted_bytes), AES.block_size)
    except ValueError:
        # Si hay un error con el padding, intentar sin desempaquetar
        decrypted_bytes = cipher.decrypt(encrypted_bytes)
        # Limitar bytes al tamaño original
        decrypted_bytes = decrypted_bytes[:img_size[0] * img_size[1] * 3]
    
    # Convertir a imagen
    decrypted_array = np.frombuffer(decrypted_bytes, dtype=np.uint8)
    decrypted_array = decrypted_array[:img_size[0] * img_size[1] * 3].reshape((img_size[1], img_size[0], 3))
    decrypted_img = Image.fromarray(decrypted_array)
    
    return decrypted_img

In [65]:
# Encriptar imagen con ECB
img_path = "pic.png"
print("Cifrando en modo ECB...")
key_ecb, _, img_size, ecb_output = encrypt_image(img_path, mode="ECB")
print(f"Imagen cifrada guardada como: {ecb_output}")

Cifrando en modo ECB...
Imagen cifrada guardada como: encrypted_ECB_pic.png


In [72]:
# Descifrar imagen con ECB
print("\nDescifrando imagen ECB...")
decrypted_ecb_img = decrypt_image_from_png(ecb_output, key_ecb, mode="ECB", img_size=img_size)
decrypted_ecb_output = "decrypted_ECB.png"
decrypted_ecb_img.save(decrypted_ecb_output)
print(f"Imagen descifrada guardada como: {decrypted_ecb_output}")


Descifrando imagen ECB...
Imagen descifrada guardada como: decrypted_ECB.png


In [67]:
# Encriptar imagen con CBC
print("\nCifrando en modo CBC...")
key_cbc, iv_cbc, img_size, cbc_output = encrypt_image(img_path, mode="CBC")
print(f"Imagen cifrada guardada como: {cbc_output}")


Cifrando en modo CBC...
Imagen cifrada guardada como: encrypted_CBC_pic.png


In [68]:
# Descifrar imagen con CBC
print("\nDescifrando imagen CBC...")
decrypted_cbc_img = decrypt_image_from_png(cbc_output, key_cbc, iv_cbc, mode="CBC", img_size=img_size)
decrypted_cbc_output = "decrypted_CBC.png"
decrypted_cbc_img.save(decrypted_cbc_output)
print(f"Imagen descifrada guardada como: {decrypted_cbc_output}")


Descifrando imagen CBC...
Imagen descifrada guardada como: decrypted_CBC.png
